In [429]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, validation_curve
from sklearn.linear_model import LogisticRegression

from scipy.stats import randint as randint
from scipy.stats import uniform

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.model_selection import cross_val_score

from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [430]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 22 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: floa

In [431]:
def preproc(df_init):
    df_preproc = df_init.copy()
    #df_preproc = df_preproc.drop(['day_of_week', 'month', '_id','emp.var.rate', 'cons.conf.idx'], axis=1)
    df_preproc = df_preproc.drop(['day_of_week', 'month', '_id'], axis=1)
       
    #df_preproc.loc[:,  ["Int'l Plan", 'VMail Plan']] = \
    #df_preproc.loc[:,  ["Int'l Plan", 'VMail Plan']].replace({'no': 0, 'yes': 1})
    
    #df_preproc.loc[:,  'Churn?'] = df_preproc.loc[:,  'Churn?'].replace({'False.': 0,
    #                                                                     'True.': 1})
    
    df_preproc = pd.get_dummies(df_preproc, columns=['job'])
    df_preproc = pd.get_dummies(df_preproc, columns=['marital'])
    df_preproc = pd.get_dummies(df_preproc, columns=['education'])
    df_preproc = pd.get_dummies(df_preproc, columns=['default'])
    df_preproc = pd.get_dummies(df_preproc, columns=['housing'])
    df_preproc = pd.get_dummies(df_preproc, columns=['loan'])
    df_preproc = pd.get_dummies(df_preproc, columns=['contact'])
    df_preproc = pd.get_dummies(df_preproc, columns=['poutcome'])
    
    #df_preproc = df_preproc[df_preproc.duration != 0]
    
    return df_preproc

In [432]:
df_preproc = df_train.pipe(preproc)
#df_preproc = df_train.copy()

In [433]:
df_preproc.describe().T

,count,mean,std,min,25%,50%,75%,max
age,24712.0,40.030188,10.462301,17.000,32.000,38.000,47.000,98.000
duration,24712.0,258.345298,261.573344,0.000,102.000,180.000,319.000,4918.000
campaign,24712.0,2.580163,2.776757,1.000,1.000,2.000,3.000,43.000
pdays,24712.0,963.558797,184.224885,0.000,999.000,999.000,999.000,999.000
previous,24712.0,0.171900,0.491511,0.000,0.000,0.000,0.000,6.000
emp.var.rate,24712.0,0.082713,1.572300,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,24712.0,93.576386,0.579548,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,24712.0,-40.516077,4.639359,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,24712.0,3.619397,1.737099,0.634,1.344,4.857,4.961,5.045
nr.employed,24712.0,5166.988111,72.516221,4963.600,5099.100,5191.000,5228.100,5228.100


In [434]:
n_splits = 5
random_state = 123
n_iter = 200 

In [435]:
# получим вектор y и матрицу признаков Х (попутно удалив колонку _id, ибо она ну совсем не нужна нам) 
y = df_preproc['target']
X = df_preproc.drop(['target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [436]:
from scipy.stats import randint as sp_randint

In [437]:
param_grid = {
    'criterion': ['gini','entropy'],
    'max_depth': randint(2, 25),
    'min_samples_leaf':  randint(1, 12),
    'class_weight': [None, 'balanced']
}

cv = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)

model = DecisionTreeClassifier(random_state=random_state)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=random_state)
random_search.fit(X, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          fit_params=None, iid=True, n_iter=200, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0F502810>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0F9A30D0>, 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [438]:
random_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 6,
 'min_samples_leaf': 6}

In [439]:
random_search.best_score_

0.9352850949357177

In [440]:
random_search.best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [441]:
model_DTC_Random = random_search.best_estimator_
predict_DTC_Random = model_DTC_Random.predict(X_test)
predict_proba_DTC_Random = model_DTC_Random.predict_proba(X_test)

In [442]:
roc_auc_DTC_Random = roc_auc_score(y_test, predict_proba_DTC_Random[:,1])
print("roc_auc_DTC_Random = ", roc_auc_DTC_Random)

roc_auc_DTC_Random =  0.9500755708438315


# line regr

In [443]:
model_LR = LogisticRegression()
model_LR.fit(X_train, y_train)
predict_LR = model_LR.predict(X_test)
predict_proba_LR = model_LR.predict_proba(X_test)

In [444]:
roc_auc_LR = roc_auc_score(y_test, predict_proba_LR[:,1])

In [445]:
roc_auc_LR

0.9246907316070184

n_splits = 5
random_state = 123
n_iter = 200 

roc_auc_DTC_Random =  0.9500755708438315

roc_auc_LR = 0.9246907316070184

----

n_splits = 7
random_state = 42
n_iter = 200


roc_auc_DTC_Random = 0.9373344725177232

In [446]:
# переполучим данные заново, чистоты для 
df_test = pd.read_csv('data/test.csv')

# почистим и преобразуем
df_test_preproc = df_test.pipe(preproc)
X_test = df_test_preproc

# и наконец сделаем предсказание
result_predict = model_DTC_Random.predict(X_test)

In [447]:
X_test.shape,result_predict.shape,result_predict

((16476, 48), (16476,), array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [448]:
df_test['target'] = result_predict
#df_test.head().T
#df_test.info()
df_test.to_csv('team5full_df_vgl.csv', sep=',', encoding='utf-8', index=False)

In [449]:
submit2kaggle = pd.DataFrame({'_id':df_test['_id'],'target':result_predict})
#submit2kaggle.head(20)
#submit2kaggle.info()
submit2kaggle.to_csv('team5result_vgl.csv', sep=',', encoding='utf-8', index=False)